In [1]:
fileUrl = 'http://dati.comune.milano.it/dataset/e8e765fc-d882-40b8-95d8-16ff3d39eb7c/resource/9c4e0776-56fc-4f3d-8a90-f4992a3be426/download/ds964_nil_wm.geojson'

In [2]:
try:
    print("Installing Folium...\n")
    !conda install -c conda-forge folium=0.5.0 --yes
    print("Folium has been successfully installed!\n")
except:
    print("ERROR: could not install Folium!\n")

Installing Folium...

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
 

In [3]:
try:
    print("Installing Geopandas...\n")
    !conda install geopandas
    print("Geopandas has been successfully installed!\n")
except:
    print("ERROR: could not install Geopandas!\n")

Installing Geopandas...

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopandas


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    json-c-0.13.1              |       h1bed415_0          70 KB
    openjpeg-2.3.0             |       h05c96fa_1         456 KB
    cligj-0.5.0                |           py36_0          12 KB
    libnetcdf-4.6.1            |       h11d0813_2         1.3 MB
    libspatialindex-1.9.3      |       he6710b0_0         3.1 MB
    libdap4-3.19.1             |       h6ec2957_0         1.5 MB
    proj4-5.2.0                |       he6710b0_1         7.0 MB
    xerces-c-3.2.2             |       h780794e_0         3.2 MB
    hdf4-4.2.13                |       h3ca952b_2         916 KB
    fiona-1.8.4                |   py36hc38cc03_0         1.0 MB
    kealib-1.4.7               |       h

In [4]:
import json
import requests
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [5]:
import folium
import geopandas as gpd

In [6]:
# read the shape files of the french departments
dep = gpd.read_file(fileUrl)
# set the coordinates to Long/Lat WGS84
dep.crs = {'init':'epsg:4326'}
# define the folium map object
M = folium.Map(location=[45.4636707, 9.1881263], zoom_start=12, tiles='Stamen Terrain')
# add the geometries
folium.GeoJson(dep.geometry.to_json()).add_to(M)
M

In [7]:
r = requests.get(fileUrl)
dictr = r.json()
df = json_normalize(dictr["features"])
df['center']=dep.geometry.centroid
df.head()

,geometry.coordinates,geometry.type,id,properties.Fonte,properties.ID_NIL,properties.NIL,properties.OBJECTID,properties.Shape_Area,properties.Shape_Length,properties.Valido_al,properties.Valido_dal,type,center
0,"[[[9.15422102515081, 45.43775166985864], [9.15...",Polygon,89,Milano 2030 - PGT Approvato,48,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,89,2.406306e+06,8723.368714,Vigente,05/02/2020,Feature,POINT (9.13726 45.43846)
1,"[[[9.10622707748691, 45.4901620817868], [9.105...",Polygon,90,Milano 2030 - PGT Approvato,64,TRENNO,90,4.896921e+05,3309.998800,Vigente,05/02/2020,Feature,POINT (9.10167 45.49282)
2,"[[[9.156361803387014, 45.48785426899354], [9.1...",Polygon,91,Milano 2030 - PGT Approvato,67,PORTELLO,91,9.096022e+05,3800.750663,Vigente,05/02/2020,Feature,POINT (9.15395 45.48449)
3,"[[[9.168034631377399, 45.522339432089304], [9....",Polygon,92,Milano 2030 - PGT Approvato,81,BOVISASCA,92,1.578028e+06,7105.469715,Vigente,05/02/2020,Feature,POINT (9.15673 45.51743)
4,"[[[9.200403184652478, 45.5284767228649], [9.20...",Polygon,93,Milano 2030 - PGT Approvato,84,PARCO NORD,93,1.532331e+06,11741.717005,Vigente,05/02/2020,Feature,POINT (9.18424 45.52351)


In [8]:
# define the folium map object
C = folium.Map(location=[45.4636707, 9.1881263], zoom_start=12, tiles='Stamen Terrain')
# add the geometries
folium.GeoJson(dep.geometry.to_json()).add_to(C)

#Creo un dataframe vuoto
data = pd.DataFrame(columns=['Neigh_Id', 'Neigh_name', 'Neig_Lat', 'Neig_long'])

for row in df.iterrows():
     row_values=row[1]
     center_point=row_values['center']
     location = [center_point.y, center_point.x]
     popup = (str(row_values['properties.ID_NIL']))
     #popup = row_values['properties.NIL']
     marker = folium.Marker(location=location, popup=popup)
     marker.add_to(C)
    
     #Creao il dataframe
     data = data.append({'Neigh_Id': row_values['properties.ID_NIL'] , 'Neigh_name': row_values['properties.NIL'], 'Neig_Lat': center_point.y, 'Neig_long':center_point.x}, ignore_index=True)
        

folium.LayerControl().add_to(C)

C

In [9]:
#Dataframe contenente i quartieri e il loro centroidi
data.head()

,Neigh_Id,Neigh_name,Neig_Lat,Neig_long
0,48,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,45.438460,9.137260
1,64,TRENNO,45.492822,9.101675
2,67,PORTELLO,45.484490,9.153947
3,81,BOVISASCA,45.517433,9.156731
4,84,PARCO NORD,45.523514,9.184235


In [10]:
#we remove the neighboroods 87 e 40 --> we already know they won't have restaurant or tourist attraction
data = data[data['Neigh_Id']!=87]
data = data[data['Neigh_Id']!=40]
data.shape

(86, 4)

# kmeans

This is the function to get the data for the venues from Foursquare

In [11]:
CLIENT_ID = '0S0V4TRZ4L30XRAN0VLDW4C3S2DBBHM1JVMSWUDJNSBDBGCJ' # your Foursquare ID
CLIENT_SECRET = 'HWL1PJYSZPYGXZBGKBWUDOE01ANCO0MMIJPEZJFBSKMNRMTY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
    
    
    
        tryagain = True
        i=0
        #more call to be sure i get the data also if the call go in error
        while tryagain is True and i<50:
            print(i) #used as check
            try:
                # create the API request URL
                url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
                    CLIENT_ID, 
                    CLIENT_SECRET, 
                    VERSION, 
                    lat, 
                    lng, 
                    radius, 
                    LIMIT)

                # make the GET request
                results = requests.get(url).json()["response"]['groups'][0]['items']

                tryagain=False
            except:
                tryagain=True
                i=i+1
        
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
milano_venues = getNearbyVenues(names=data['Neigh_name'],
                                   latitudes=data['Neig_Lat'],
                                   longitudes=data['Neig_long']
                                  )

RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO
0
TRENNO
0
PORTELLO
0
BOVISASCA
0
PARCO NORD
0
FIGINO
0
LORETO - CASORETTO - NOLO
0
QUARTO OGGIARO - VIALBA - MUSOCCO
0
ISOLA
0
QUARTO CAGNINO
0
STADIO - IPPODROMI
0
QUINTO ROMANO
0
DUOMO
0
GUASTALLA
0
SAN SIRO
0
COMASINA
0
TIBALDI
0
GRECO - SEGNANO
0
DE ANGELI - MONTE ROSA
0
FARINI
0
BRUZZANO
0
QT 8
0
STEPHENSON
0
CANTALUPA
0
QUINTOSOLE
0
PARCO SEMPIONE
0
BARONA
0
VILLAPIZZONE - CAGNOLA - BOLDINASCO
0
PARCO BOSCO IN CITTA'
0
GORLA - PRECOTTO
0
NIGUARDA - CA' GRANDA - PRATO CENTENARO - Q.RE FULVIO TESTI
0
TRIULZO SUPERIORE
0
PTA ROMANA
0
TALIEDO - MORSENCHIO - Q.RE FORLANINI
0
PORTA TICINESE - CONCA DEL NAVIGLIO
0
TRE TORRI
0
MORIVIONE
0
VIGENTINO - Q.RE FATIMA
0
BICOCCA
0
ORTOMERCATO
0
LODI - CORVETTO
0
MUGGIANO
0
PORTA TICINESE - CONCHETTA
0
UMBRIA - MOLISE - CALVAIRATE
0
ROSERIO
0
Q.RE GALLARATESE - Q.RE SAN LEONARDO - LAMPUGNANO
0
MONLUE' - PONTE LAMBRO
0
PADOVA - TURRO - CRESCENZAGO
0
GRATOSOGLIO - Q.RE MISSAGLIA - Q.RE TERRAZZE
0
POR

In [13]:
print(milano_venues.shape)
milano_venues.head()

(2219, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,45.438460,9.137260,Simply Market,45.440557,9.135820,Supermarket
1,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,45.438460,9.137260,Pizzeria Bis,45.441764,9.139459,Pizza Place
2,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,45.438460,9.137260,Centro Sportivo Parenzo,45.442730,9.138951,Gym / Fitness Center
3,TRENNO,45.492822,9.101675,La Ratera,45.491371,9.099451,Pub
4,TRENNO,45.492822,9.101675,Forno Trenno,45.491774,9.101133,Bakery


In [14]:
milano_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
ADRIANO,14,14,14,14,14,14
AFFORI,15,15,15,15,15,15
BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,8,8,8,8,8,8
BANDE NERE,19,19,19,19,19,19
BARONA,4,4,4,4,4,4
BICOCCA,43,43,43,43,43,43
BOVISA,29,29,29,29,29,29
BOVISASCA,10,10,10,10,10,10
BRERA,97,97,97,97,97,97


In [15]:
tr = milano_venues.groupby('Neighborhood').count()
tr[tr['Neighborhood Latitude']<3]

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
MAGGIORE - MUSOCCO - CERTOSA,1,1,1,1,1,1
PARCO DEI NAVIGLI,1,1,1,1,1,1
PARCO DELLE ABBAZIE,2,2,2,2,2,2
QUINTOSOLE,2,2,2,2,2,2


Let's find out how many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(milano_venues['Venue Category'].unique())))

There are 258 uniques categories.


Analyze Each Neighborhood

In [17]:
# one hot encoding
milano_onehot = pd.get_dummies(milano_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
milano_onehot['Neighborhood'] = milano_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [milano_onehot.columns[-1]] + list(milano_onehot.columns[:-1])
milano_onehot = milano_onehot[fixed_columns]

milano_onehot.head()

,Yoga Studio,Accessories Store,Adult Education Center,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,Arcade,Argentinian Restaurant,...,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
milano_grouped = milano_onehot.groupby('Neighborhood').mean().reset_index()
milano_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Education Center,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,Arcade,...,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Women's Store
0,ADRIANO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AFFORI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BANDE NERE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BARONA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


First, let's write a function to sort the venues in descending order.

In [19]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe with the top 10 venues for each neighborhood.

In [56]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = milano_grouped['Neighborhood']

for ind in np.arange(milano_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(milano_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ADRIANO,Supermarket,Italian Restaurant,Trattoria/Osteria,Plaza,Pizza Place,Pharmacy,Electronics Store,Park,Soccer Field,Bistro
1,AFFORI,Pizza Place,Park,Café,Italian Restaurant,Health Food Store,Pool Hall,Restaurant,Supermarket,Gym,Fried Chicken Joint
2,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,Italian Restaurant,Pizza Place,Bar,Park,Soccer Field,Café,Women's Store,Flea Market,Fast Food Restaurant,Fish & Chips Shop
3,BANDE NERE,Restaurant,Pizza Place,Hotel,Bus Stop,Hobby Shop,Pharmacy,Café,Athletics & Sports,Diner,Pub
4,BARONA,Athletics & Sports,Tennis Stadium,Soccer Field,Food Court,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Women's Store


Run k-means to cluster the neighborhood into 5 clusters.

In [57]:
# set number of clusters
kclusters = 9

milano_grouped_clustering = milano_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(milano_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,6,ADRIANO,Supermarket,Italian Restaurant,Trattoria/Osteria,Plaza,Pizza Place,Pharmacy,Electronics Store,Park,Soccer Field,Bistro
1,0,AFFORI,Pizza Place,Park,Café,Italian Restaurant,Health Food Store,Pool Hall,Restaurant,Supermarket,Gym,Fried Chicken Joint
2,1,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,Italian Restaurant,Pizza Place,Bar,Park,Soccer Field,Café,Women's Store,Flea Market,Fast Food Restaurant,Fish & Chips Shop
3,5,BANDE NERE,Restaurant,Pizza Place,Hotel,Bus Stop,Hobby Shop,Pharmacy,Café,Athletics & Sports,Diner,Pub
4,2,BARONA,Athletics & Sports,Tennis Stadium,Soccer Field,Food Court,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Women's Store


Let's create a new dataframe that includes the top 10 venues for each neighborhood.

In [58]:
milano_merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
milano_merged = milano_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neigh_name')

milano_merged.head(5) 

,Neigh_Id,Neigh_name,Neig_Lat,Neig_long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,48,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,45.438460,9.137260,0,Pizza Place,Supermarket,Gym / Fitness Center,Women's Store,Fish & Chips Shop,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flower Shop
1,64,TRENNO,45.492822,9.101675,0,Pizza Place,Sandwich Place,Plaza,Park,Soccer Field,Bakery,Ice Cream Shop,Adult Education Center,Pub,Food & Drink Shop
2,67,PORTELLO,45.484490,9.153947,5,Italian Restaurant,Café,Japanese Restaurant,Hotel,Supermarket,Ice Cream Shop,Gym,Snack Place,Seafood Restaurant,Cheese Shop
3,81,BOVISASCA,45.517433,9.156731,6,Soccer Field,Italian Restaurant,Restaurant,Bus Stop,Shopping Plaza,Supermarket,Park,Art Gallery,Gym / Fitness Center,Dive Bar
4,84,PARCO NORD,45.523514,9.184235,6,Women's Store,Football Stadium,Supermarket,Playground,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market


Finally, let's visualize the resulting clusters

In [59]:
# create map
map_clusters = folium.Map(location=[45.4636707, 9.1881263], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(milano_merged['Neig_Lat'], milano_merged['Neig_long'], milano_merged['Neigh_name'],  milano_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [71]:
milano_merged.loc[milano_merged['Cluster Labels'] == 0, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,Pizza Place,Supermarket,Gym / Fitness Center,Women's Store,Fish & Chips Shop,Event Space,Farm,Farmers Market,Fast Food Restaurant,Flower Shop
1,TRENNO,Pizza Place,Sandwich Place,Plaza,Park,Soccer Field,Bakery,Ice Cream Shop,Adult Education Center,Pub,Food & Drink Shop
5,FIGINO,Bakery,Volleyball Court,Stadium,Women's Store,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food & Drink Shop
15,COMASINA,Bus Station,Café,Gym / Fitness Center,Metro Station,Fast Food Restaurant,Pizza Place,Bar,General Entertainment,Convenience Store,Recreation Center
30,NIGUARDA - CA' GRANDA - PRATO CENTENARO - Q.RE...,Pizza Place,Sushi Restaurant,Bar,Hotel,Theater,Women's Store,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
33,TALIEDO - MORSENCHIO - Q.RE FORLANINI,Pizza Place,Tram Station,Tennis Court,Spa,Recording Studio,Hotel,Flea Market,Bus Stop,Shipping Store,Fast Food Restaurant
38,VIGENTINO - Q.RE FATIMA,Pizza Place,Tram Station,Betting Shop,Café,Supermarket,Italian Restaurant,Seafood Restaurant,Food,Food & Drink Shop,Food Court
41,LODI - CORVETTO,Nightclub,Pizza Place,Soccer Field,Italian Restaurant,Hotel,Vegetarian / Vegan Restaurant,American Restaurant,Café,Sports Club,Seafood Restaurant
47,Q.RE GALLARATESE - Q.RE SAN LEONARDO - LAMPUGNANO,Pizza Place,Fast Food Restaurant,Café,Fried Chicken Joint,Church,Asian Restaurant,Ice Cream Shop,Bookstore,Clothing Store,Fish & Chips Shop
52,FORZE ARMATE,Athletics & Sports,Dance Studio,Soccer Field,Supermarket,Bakery,Pizza Place,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop


In [72]:
milano_merged.loc[milano_merged['Cluster Labels'] == 1, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,LORETO - CASORETTO - NOLO,Italian Restaurant,Japanese Restaurant,Ice Cream Shop,Nightclub,Trattoria/Osteria,Pharmacy,Music Venue,Pub,Park,Chinese Restaurant
16,TIBALDI,Italian Restaurant,Japanese Restaurant,Pizza Place,Supermarket,Gym,Café,Bus Stop,Sushi Restaurant,Steakhouse,Brewery
21,QT 8,Italian Restaurant,Sandwich Place,Track,Soccer Field,Park,Restaurant,Fish & Chips Shop,Farm,Farmers Market,Fast Food Restaurant
40,ORTOMERCATO,Italian Restaurant,Farmers Market,Music Venue,Park,Soccer Field,Pizza Place,Flower Shop,Flea Market,Fast Food Restaurant,Fish & Chips Shop
44,UMBRIA - MOLISE - CALVAIRATE,Italian Restaurant,Pizza Place,Tram Station,Convenience Store,Sandwich Place,Sushi Restaurant,Supermarket,Beer Bar,Burger Joint,Bus Stop
51,PORTA MAGENTA,Italian Restaurant,Café,Supermarket,Hardware Store,Pizza Place,Restaurant,Pharmacy,Chinese Restaurant,Bus Stop,Bistro
54,CHIARAVALLE,Fruit & Vegetable Store,Italian Restaurant,Convenience Store,Restaurant,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
63,CIMIANO - ROTTOLE - Q.RE FELTRE,Performing Arts Venue,Dog Run,Italian Restaurant,Park,Bookstore,Soccer Field,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
65,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,Italian Restaurant,Pizza Place,Bar,Park,Soccer Field,Café,Women's Store,Flea Market,Fast Food Restaurant,Fish & Chips Shop
66,PARCO FORLANINI - CAVRIANO,Italian Restaurant,Art Gallery,Rock Club,Park,Steakhouse,Nightclub,Hotel Bar,Hotel,Food Truck,Food Court


In [73]:
milano_merged.loc[milano_merged['Cluster Labels'] == 2, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,QUINTO ROMANO,Hotel,Burger Joint,Soccer Field,Playground,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop
22,STEPHENSON,Hotel,Restaurant,Supermarket,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Women's Store
26,BARONA,Athletics & Sports,Tennis Stadium,Soccer Field,Food Court,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Women's Store
28,PARCO BOSCO IN CITTA',Flower Shop,Hotel,Campground,Park,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Women's Store,Event Space
48,MONLUE' - PONTE LAMBRO,Airport Terminal,Hotel Bar,Breakfast Spot,Moroccan Restaurant,Hotel,Food & Drink Shop,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop


In [74]:
milano_merged.loc[milano_merged['Cluster Labels'] == 3, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
74,MAGGIORE - MUSOCCO - CERTOSA,Trattoria/Osteria,Women's Store,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food,Emilia Restaurant


In [75]:
milano_merged.loc[milano_merged['Cluster Labels'] == 4, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
82,PARCO DEI NAVIGLI,Movie Theater,Women's Store,Fruit & Vegetable Store,Fried Chicken Joint,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food


In [76]:
milano_merged.loc[milano_merged['Cluster Labels'] == 5, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,PORTELLO,Italian Restaurant,Café,Japanese Restaurant,Hotel,Supermarket,Ice Cream Shop,Gym,Snack Place,Seafood Restaurant,Cheese Shop
8,ISOLA,Italian Restaurant,Café,Cocktail Bar,Pizza Place,Ice Cream Shop,Pub,Burger Joint,Japanese Restaurant,Ramen Restaurant,Chinese Restaurant
9,QUARTO CAGNINO,Café,Italian Restaurant,Pizza Place,Park,Pub,Athletics & Sports,Bus Stop,Campanian Restaurant,Supermarket,Bed & Breakfast
10,STADIO - IPPODROMI,Soccer Stadium,Racecourse,Stadium,Bed & Breakfast,Palace,Bar,Snack Place,Golf Course,Outdoor Sculpture,Athletics & Sports
12,DUOMO,Hotel,Italian Restaurant,Plaza,Monument / Landmark,Ice Cream Shop,Boutique,Coffee Shop,Bakery,Art Museum,Art Gallery
13,GUASTALLA,Pizza Place,Italian Restaurant,Sandwich Place,Furniture / Home Store,Sporting Goods Shop,Hotel,Bistro,Ice Cream Shop,Coffee Shop,Japanese Restaurant
17,GRECO - SEGNANO,Italian Restaurant,Pizza Place,Café,Chinese Restaurant,Pet Store,Cocktail Bar,Coffee Shop,Tunnel,Steakhouse,Plaza
18,DE ANGELI - MONTE ROSA,Italian Restaurant,Café,Pizza Place,Hotel,Plaza,Bar,Asian Restaurant,Piadineria,Burger Joint,Brewery
19,FARINI,Café,Hotel,Italian Restaurant,Pizza Place,Bakery,Outlet Store,Electronics Store,Supermarket,Tram Station,Bar
20,BRUZZANO,Gym / Fitness Center,Italian Restaurant,Diner,Hotel,Bakery,Train Station,Flea Market,Event Space,Farm,Farmers Market


In [77]:
milano_merged.loc[milano_merged['Cluster Labels'] == 6, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,BOVISASCA,Soccer Field,Italian Restaurant,Restaurant,Bus Stop,Shopping Plaza,Supermarket,Park,Art Gallery,Gym / Fitness Center,Dive Bar
4,PARCO NORD,Women's Store,Football Stadium,Supermarket,Playground,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
7,QUARTO OGGIARO - VIALBA - MUSOCCO,Supermarket,Park,Pizza Place,Women's Store,Flower Shop,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food
14,SAN SIRO,Bus Stop,Supermarket,Chinese Restaurant,Italian Restaurant,Tram Station,Flea Market,Movie Theater,Café,Farmers Market,Sushi Restaurant
42,MUGGIANO,Juice Bar,Airport,Soccer Field,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Flower Shop,Food Court
50,GRATOSOGLIO - Q.RE MISSAGLIA - Q.RE TERRAZZE,Bus Stop,Italian Restaurant,Light Rail Station,Tram Station,Supermarket,Food,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market
58,LAMBRATE - ORTICA,Nightclub,Café,Plaza,Buffet,Electronics Store,Steakhouse,Theater,Supermarket,Bus Stop,Flower Shop
59,STADERA - CHIESA ROSSA - Q.RE TORRETTA - CONCA...,Tram Station,Italian Restaurant,Supermarket,Cocktail Bar,Gym,Flea Market,Event Space,Farm,Farmers Market,Fast Food Restaurant
71,MONCUCCO - SAN CRISTOFORO,Supermarket,Italian Restaurant,Café,Gym,Ice Cream Shop,Art Gallery,Gastropub,Bed & Breakfast,IT Services,Rock Club
83,PAGANO,Supermarket,Plaza,Cocktail Bar,Burger Joint,Bar,Asian Restaurant,Cosmetics Shop,Italian Restaurant,Restaurant,Park


In [78]:
milano_merged.loc[milano_merged['Cluster Labels'] == 7, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
24,QUINTOSOLE,Italian Restaurant,Flower Shop,Women's Store,Emilia Restaurant,Fountain,Football Stadium,Food Truck,Food Court,Food & Drink Shop,Food


In [79]:
milano_merged.loc[milano_merged['Cluster Labels'] == 8, milano_merged.columns[[1] + list(range(5, milano_merged.shape[1]))]]

,Neigh_name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,PARCO DELLE ABBAZIE,Stables,Restaurant,Women's Store,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Food & Drink Shop


# Is tourist?

In [34]:
milano_venues[milano_venues['Venue Category'].str.contains("Monument")]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
205,DUOMO,45.463707,9.186948,Palazzo della Ragione,45.464792,9.187785,Monument / Landmark
207,DUOMO,45.463707,9.186948,Monumento A Vittorio Emanuele II,45.464190,9.189458,Monument / Landmark
212,DUOMO,45.463707,9.186948,Galleria Vittorio Emanuele II,45.465577,9.190024,Monument / Landmark
228,DUOMO,45.463707,9.186948,Statua a Leonardo da Vinci,45.466970,9.190010,Monument / Landmark
233,DUOMO,45.463707,9.186948,L.O.V.E. Maurizio Cattelan,45.464834,9.183314,Monument / Landmark
277,GUASTALLA,45.463219,9.201891,Rotonda della Besana,45.459985,9.205096,Monument / Landmark
484,PARCO SEMPIONE,45.474131,9.176251,Arco della Pace,45.475728,9.172447,Monument / Landmark
631,PTA ROMANA,45.450400,9.205141,Porta Romana,45.451659,9.202814,Monument / Landmark
711,PORTA TICINESE - CONCA DEL NAVIGLIO,45.450475,9.181311,Porta Ticinese (Pusterla),45.452720,9.179876,Monument / Landmark
1394,GIARDINI P.TA VENEZIA,45.474564,9.200231,Porta Venezia,45.474310,9.204898,Monument / Landmark


In [35]:
milano_venues[milano_venues['Venue Category'].str.contains("Art")]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
53,BOVISASCA,45.517433,9.156731,Arte Zara,45.517142,9.154076,Art Gallery
141,ISOLA,45.490894,9.189617,O' Artoteca,45.487015,9.186574,Performing Arts Venue
208,DUOMO,45.463707,9.186948,Pinacoteca Ambrosiana,45.463768,9.186251,Art Museum
216,DUOMO,45.463707,9.186948,Museo del Novecento - Arengario,45.463413,9.190245,Art Museum
222,DUOMO,45.463707,9.186948,Palazzo Reale,45.462960,9.191348,Art Gallery
229,DUOMO,45.463707,9.186948,Gallerie d'Italia,45.467183,9.190056,Art Gallery
388,TIBALDI,45.440348,9.180459,Centro Asteria,45.436901,9.176465,Art Museum
482,PARCO SEMPIONE,45.474131,9.176251,La Triennale di Milano,45.472243,9.173507,Art Museum
539,VILLAPIZZONE - CAGNOLA - BOLDINASCO,45.497419,9.143523,Masada,45.495910,9.142995,Performing Arts Venue
759,PORTA TICINESE - CONCA DEL NAVIGLIO,45.450475,9.181311,Auditorium di Milano,45.446420,9.179187,Performing Arts Venue


In [36]:
milano_venues[milano_venues['Venue Category'].str.contains("Museum")]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
109,ISOLA,45.490894,9.189617,Fonderia Napoleonica Eugenia,45.493050,9.187788,Museum
208,DUOMO,45.463707,9.186948,Pinacoteca Ambrosiana,45.463768,9.186251,Art Museum
216,DUOMO,45.463707,9.186948,Museo del Novecento - Arengario,45.463413,9.190245,Art Museum
388,TIBALDI,45.440348,9.180459,Centro Asteria,45.436901,9.176465,Art Museum
482,PARCO SEMPIONE,45.474131,9.176251,La Triennale di Milano,45.472243,9.173507,Art Museum
489,PARCO SEMPIONE,45.474131,9.176251,Musei Castello Sforzesco,45.470564,9.179293,Museum
793,PORTA TICINESE - CONCA DEL NAVIGLIO,45.450475,9.181311,Museo Diocesano,45.454827,9.181330,Art Museum
1404,GIARDINI P.TA VENEZIA,45.474564,9.200231,Palazzo Dugnani,45.474448,9.196929,Art Museum
1416,GIARDINI P.TA VENEZIA,45.474564,9.200231,Palazzo Della Permanente,45.476643,9.195407,Art Museum
1599,BUENOS AIRES - PORTA VENEZIA - PORTA MONFORTE,45.477025,9.214489,Casa-Museo Boschi di Stefano,45.478986,9.211562,Art Museum


In [37]:
milano_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Education Center,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,Arcade,...,Turkish Restaurant,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Women's Store
0,ADRIANO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AFFORI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BANDE NERE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BARONA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
milano_sum = milano_onehot.groupby('Neighborhood').mean().reset_index()
museum = milano_sum.loc[ : , list(milano_sum.columns[milano_sum.columns.str.contains("Museum")])]
art = milano_sum.loc[ : , list(milano_sum.columns[milano_sum.columns.str.contains("Art")])]
monument = milano_sum.loc[ : , list(milano_sum.columns[milano_sum.columns.str.contains("Monument")])]
milano_sum['Tourist']=museum.sum(axis = 1) + art.sum(axis = 1)+monument.sum(axis = 1)
# other = milano_sum.loc[ : , :]
# milano_sum['Other']= other.sum(axis = 1) - milano_sum['Tourist']
milano_sum.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Education Center,African Restaurant,Airport,Airport Terminal,American Restaurant,Amphitheater,Arcade,...,Tuscan Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Warehouse Store,Wine Bar,Wine Shop,Women's Store,Tourist
0,ADRIANO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,AFFORI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BANDE NERE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BARONA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
milan_tourist= milano_sum.loc[ : , ['Neighborhood','Tourist']]
milan_tourist.head()

,Neighborhood,Tourist
0,ADRIANO,0.0
1,AFFORI,0.0
2,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,0.0
3,BANDE NERE,0.0
4,BARONA,0.0


In [40]:
# set number of clusters
kclusters = 2

milan_elab = milan_tourist.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(milan_elab)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:2] 

milan_tourist.insert(0, 'Cluster Labels', kmeans.labels_)
milan_tourist.head(50)

,Cluster Labels,Neighborhood,Tourist
0,0,ADRIANO,0.000000
1,0,AFFORI,0.000000
2,0,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,0.000000
3,0,BANDE NERE,0.000000
4,0,BARONA,0.000000
5,0,BICOCCA,0.000000
6,0,BOVISA,0.000000
7,1,BOVISASCA,0.100000
8,1,BRERA,0.072165
9,0,BRUZZANO,0.000000


In [41]:
milan_tourist.sort_values(by=['Tourist'], ascending=False).head(13)

,Cluster Labels,Neighborhood,Tourist
50,1,PARCO FORLANINI - CAVRIANO,0.166667
14,1,CIMIANO - ROTTOLE - Q.RE FELTRE,0.166667
54,1,PORTA GENOVA,0.162791
20,1,DUOMO,0.154930
7,1,BOVISASCA,0.100000
26,1,GIARDINI P.TA VENEZIA,0.092784
37,1,MAGENTA - S. VITTORE,0.088235
52,1,PARCO SEMPIONE,0.076923
8,1,BRERA,0.072165
57,1,PORTA TICINESE - CONCHETTA,0.066667


In [42]:
milano_assembly = data
milano_assembly=milano_assembly.join(milan_tourist.set_index('Neighborhood'), on='Neigh_name')

milano_assembly.head()

,Neigh_Id,Neigh_name,Neig_Lat,Neig_long,Cluster Labels,Tourist
0,48,RONCHETTO SUL NAVIGLIO - Q.RE LODOVICO IL MORO,45.438460,9.137260,0,0.0
1,64,TRENNO,45.492822,9.101675,0,0.0
2,67,PORTELLO,45.484490,9.153947,0,0.0
3,81,BOVISASCA,45.517433,9.156731,1,0.1
4,84,PARCO NORD,45.523514,9.184235,0,0.0


In [43]:
# create map
map_clusters_tou = folium.Map(location=[45.4636707, 9.1881263], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(milano_assembly['Neig_Lat'], milano_assembly['Neig_long'], milano_assembly['Neigh_name'],  milano_assembly['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters_tou)
       
map_clusters_tou

# Result

In [81]:
result = milan_tourist
result.rename(columns={'Cluster Labels':'Is_Tourist'}, inplace=True)

result=result.join(milano_merged.set_index('Neigh_name'), on='Neighborhood')

result.head()

,Is_Tourist,Neighborhood,Tourist,Neigh_Id,Neig_Lat,Neig_long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,ADRIANO,0.0,17,45.514093,9.248356,6,Supermarket,Italian Restaurant,Trattoria/Osteria,Plaza,Pizza Place,Pharmacy,Electronics Store,Park,Soccer Field,Bistro
1,0,AFFORI,0.0,80,45.513926,9.171293,0,Pizza Place,Park,Café,Italian Restaurant,Health Food Store,Pool Hall,Restaurant,Supermarket,Gym,Fried Chicken Joint
2,0,BAGGIO - Q.RE DEGLI OLMI - Q.RE VALSESIA,0.0,55,45.459598,9.087214,1,Italian Restaurant,Pizza Place,Bar,Park,Soccer Field,Café,Women's Store,Flea Market,Fast Food Restaurant,Fish & Chips Shop
3,0,BANDE NERE,0.0,52,45.459914,9.139080,5,Restaurant,Pizza Place,Hotel,Bus Stop,Hobby Shop,Pharmacy,Café,Athletics & Sports,Diner,Pub
4,0,BARONA,0.0,46,45.432353,9.156192,2,Athletics & Sports,Tennis Stadium,Soccer Field,Food Court,Flea Market,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Women's Store


In [82]:
good = result[result['Is_Tourist']==1]
good = good[good['Cluster Labels']==5]
good.head(15)

,Is_Tourist,Neighborhood,Tourist,Neigh_Id,Neig_Lat,Neig_long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,1,BRERA,0.072165,2,45.474252,9.188157,5,Italian Restaurant,Ice Cream Shop,Japanese Restaurant,Hotel,Wine Bar,Pizza Place,Café,Bakery,Restaurant,Dessert Shop
20,1,DUOMO,0.154930,1,45.463707,9.186948,5,Hotel,Italian Restaurant,Plaza,Monument / Landmark,Ice Cream Shop,Boutique,Coffee Shop,Bakery,Art Museum,Art Gallery
26,1,GIARDINI P.TA VENEZIA,0.092784,3,45.474564,9.200231,5,Italian Restaurant,Hotel,Café,Gay Bar,Sushi Restaurant,Cocktail Bar,Art Gallery,Bar,Plaza,Bookstore
37,1,MAGENTA - S. VITTORE,0.088235,7,45.464560,9.170394,5,Italian Restaurant,Café,Plaza,Sandwich Place,Ice Cream Shop,Hotel,Historic Site,Sushi Restaurant,Pastry Shop,American Restaurant
52,1,PARCO SEMPIONE,0.076923,8,45.474131,9.176251,5,Cocktail Bar,Lounge,Wine Bar,Café,Pizza Place,Italian Restaurant,Pub,Nightclub,Bookstore,Gastropub
54,1,PORTA GENOVA,0.162791,50,45.453967,9.161885,5,Italian Restaurant,Art Gallery,Boutique,Café,Pizza Place,Hotel,Performing Arts Venue,Burger Joint,Sandwich Place,Bistro
57,1,PORTA TICINESE - CONCHETTA,0.066667,44,45.448517,9.169242,5,Italian Restaurant,Cocktail Bar,Café,Bar,Art Gallery,Tram Station,Gym,Trattoria/Osteria,Indian Restaurant,Chinese Restaurant
